In [1]:
import numpy as np
import pickle

In [76]:
# Open files
glove_mat = np.load("./data/glove_6B_50d.npy")
words = np.load("./data/words_6B_50d.npy")
with open("./data/word_to_ind_6B_50d.pkl", "rb") as f:
    word_to_ind = pickle.load(f)

In [3]:
# Input (Question)
given_f, given_s, ques_f = "father", "mother", "brother"
given_s_opts = ["grandmother", "grandfather", "daughter", "sister", "mother"]

In [17]:
# Calculate difference vectors
given_f_v = glove_mat[word_to_ind[given_f]]
given_s_v = glove_mat[word_to_ind[given_s]]
ques_f_v = glove_mat[word_to_ind[ques_f]]

given_diff = given_s_v - given_f_v

# max_sim, max_i = 0, 0

# for i, opt in enumerate(given_s_opts):
#     opt_v = glove_mat[word_to_ind[opt]]
#     diff_vec = opt_v - ques_f_v
#     sim = given_diff @ diff_vec / np.linalg.norm(given_diff) / np.linalg.norm(diff_vec)
#     print(opt, round(sim, 4))
#     if (sim > max_sim): max_sim, max_i = sim, i

diff_mat = glove_mat - ques_f_v
print(glove_mat[0][1], ques_f_v[1], diff_mat[0][1])
diff_mat_norms = np.linalg.norm(diff_mat, axis=1, keepdims=True)
print(diff_mat.shape, diff_mat_norms.shape)

# print(given_diff)
given_diff = np.reshape(given_diff, (-1, 1)) # to col vec
# print(given_diff)
given_diff_norm = np.linalg.norm(given_diff, keepdims=True)
print(given_diff.shape, given_diff_norm.shape)

-0.12123 0.12911 -0.25034
(1917495, 300) (1917495, 1)
[ 5.4736200e-01  1.7924000e-01  2.0883640e-01  1.0410000e-02
  4.4846000e-01 -2.9254400e-01 -1.2010000e-01  6.3940000e-02
 -9.3680000e-02 -3.9694000e-01  2.8933000e-01 -2.1917400e-01
  2.5620400e-01 -6.9700000e-02 -3.9457000e-01 -1.3850700e-01
 -1.5240000e-02  6.1990000e-02 -2.0984700e-01  2.6920000e-03
 -9.3830000e-02 -3.0800000e-03  5.3433300e-01 -8.6500000e-02
 -1.4387000e-01  2.6825700e-01  3.8126000e-01 -1.5600000e-03
 -1.5728700e-01 -3.3187100e-01 -1.9283900e-01 -2.7704000e-01
  6.1055100e-01  2.1620000e-01  4.1405000e-01  1.3012000e-01
  4.2722000e-01 -4.2816800e-01 -4.3840000e-02 -4.3878900e-01
 -1.2391000e-01 -1.6183000e-01 -6.5890000e-02 -3.2598300e-01
  8.6480000e-02  3.4201200e-01 -2.2823300e-01 -7.0435000e-02
 -4.5984000e-01 -1.9876370e-01  4.3922000e-02  2.9632000e-01
  1.8973000e-01 -2.9150000e-02 -1.5175000e-01  4.1370000e-01
  2.6388000e-01  1.0546500e-01  4.7113000e-01 -4.8377000e-01
  4.8276000e-02  1.8465900e-01 

In [9]:

# Calculate cosine similarities between difference vectors
diff_mat_sims = diff_mat @ given_diff
print(diff_mat_sims.shape)
diff_mat_sims = diff_mat_sims / diff_mat_norms / given_diff_norm
print(diff_mat_sims.shape)

    

(1917495, 1)
(1917495, 1)


In [13]:
# Find max similarity
max_sim = np.nanmax(diff_mat_sims)
max_i = np.nanargmax(diff_mat_sims)

print(max_i, max_sim)
print("Prediction:", words[max_i], round(max_sim, 4))

929 0.6085033239907418
Prediction: mother 0.6085


In [18]:
# Testing answer
sister_diff = glove_mat[word_to_ind["sister"]] - glove_mat[word_to_ind["brother"]]
sister_sim = sister_diff @ given_diff / np.linalg.norm(sister_diff) / np.linalg.norm(given_diff)
print(sister_sim)

[0.5028397]


In [77]:
# Testing distance between synonyms and antonyms
syn = np.linalg.norm(glove_mat[word_to_ind["happy"]] - glove_mat[word_to_ind["joyful"]])
ant = np.linalg.norm(glove_mat[word_to_ind["happy"]] - glove_mat[word_to_ind["sad"]])
print(syn, ant)

4.221551950609508 3.8399498989360525


In [84]:
# Closest neighbor approach
vec = glove_mat[word_to_ind["teach"]] - glove_mat[word_to_ind["teacher"]] + glove_mat[word_to_ind["writer"]]
neighbors = sorted(words, key=lambda word: np.linalg.norm(vec - glove_mat[word_to_ind[word]]))
print(neighbors[:10])

['writers', 'writer', 'fiction', 'author', 'authors', 'writes', 'writing', 'book', 'editors', 'creators']


In [85]:
# Print distances
for i in range(11):
    dist = np.linalg.norm(vec - glove_mat[word_to_ind[neighbors[i]]])
    print(dist)
print(np.linalg.norm(vec - glove_mat[word_to_ind["angrier"]]))

3.436503592798781
3.8912356527527256
4.25787288694272
4.28108676885505
4.3490379799092915
4.444380809626683
4.557489395703837
4.571841281479966
4.591737090175452
4.603339046465742
4.638593005327816
7.985134030422456


In [81]:

# Distribution of pairwise distances
n = 200
indices = np.random.randint(len(words), size=n)
avg = 0
std_dev = 0
count = 0
min_dist, max_dist = 100, 0
for i in range(n):
    for j in range(i + 1, n):
        dist = np.linalg.norm(glove_mat[indices[i]] - glove_mat[indices[j]])
        avg += dist
        count += 1
        min_dist, max_dist = min(min_dist, dist), max(max_dist, dist)
avg /= count

for i in range(n):
    for j in range(n):
        dist = np.linalg.norm(glove_mat[indices[i]] - glove_mat[indices[j]])
        std_dev += (dist - avg)**2
std_dev /= count
std_dev = std_dev**(1/2)

print(avg, std_dev, min_dist, max_dist)


6.056644592597328 1.8199036142600775 2.3182959449967315 13.21343975129512
